In [2]:
import logging
import os, sys
if not "../" in sys.path:
    sys.path.insert(0, "../")

In [3]:
sys.path

['../',
 '/mnt/computerome/home/projects/dtu_00009/people/henspi/git/Screener/notebooks',
 '/home/henrik/anaconda3/lib/python310.zip',
 '/home/henrik/anaconda3/lib/python3.10',
 '/home/henrik/anaconda3/lib/python3.10/lib-dynload',
 '',
 '/home/henrik/anaconda3/lib/python3.10/site-packages']

In [5]:
from scripts.qsub_mapquantifier import QuantifierMap

In [6]:
import inspect

In [4]:
"abc.gz"[:-3]

'abc'

In [84]:
os.path.dirname("a/b/c")

'a/b'

In [82]:
class qsub_base:
    working_dir = "/home/projects/dtu_00009/people/henspi/git/AntibioticaScreening/project"
    
    
    @property
    def log_setup(self):
        setup = dict(
            name = self.__class__.__name__,
            level = logging.DEBUG,
            log_file = None
        )    
        return setup
    @property
    def log(self):
        if not hasattr(self, "_log"):
            setup = self.log_setup
            logger = logging.getLogger(setup["name"])
            
            #remove old handlers:
            while logger.handlers:
                logger.removeHandler(logger.handlers[0])
            
            logger.setLevel(setup["level"])
            
            formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
            stream_handler = logging.StreamHandler()
            stream_handler.setFormatter(formatter)
            stream_handler.setLevel(setup["level"])
            logger.addHandler(stream_handler)
            
            if setup["log_file"]:
                file_handler = logging.FileHandler(setup["log_file"])
                file_handler.setFormatter(formatter)
                file_handler.setLevel(setup["level"])
                logger.addHandler(file_handler)
                logger.debug("logfile at -> "+setup["log_file"])
            self._log = logger
        return self._log
  
    @property
    def default_qsub_requirement(self) -> dict:
        self.log.warning("Using default qsub_base compute requirements - set property: default_qsub_requirement in class")
        args = dict(
            modules = "tools anaconda3/2021.11 ",
            runtime = 30,
            cores = 5,
            ram=20,
        )
        return args

    def gen_qsub_args(self, jobtag: str=None, **kwargs):
        jobname = self.__class__.__name__
        if jobtag:
            jobname = "_".join([jobname, jobtag])
        qsub_args = self.default_qsub_requirement
        qsub_args.update(dict(
            directory = self.working_dir,
            group="dtu_00009",
            jobname=jobname,
            output = os.path.join("logs", jobname+ "_stdout"),
            error = os.path.join("logs", jobname+ "_stderr")
        ))
        qsub_args.update(kwargs)
        return qsub_args
    
    def generate_syscall(self, **kwargs) -> str:
        raise NotImplementedError
        
    def preflight(self, **kwargs) -> None:
        raise NotImplementedError
        
    def output_exist(self, **kwargs) -> bool:
        raise NotImplementedError
    
    def is_succes(self) -> bool:
        raise NotImplementedError


In [91]:
import re
dir_p = "../data/simulated_data/camisim/0_1GB/2022.09.08_08.12.03_sample_0/reads/"
files = os.listdir(dir_p)
reads1 = sorted([x for x in files if re.search("Genome\d1.fq.gz", x)])
reads2 = sorted([x for x in files if re.search("Genome\d2.fq.gz", x)])

In [92]:
reads1

['Genome11.fq.gz',
 'Genome21.fq.gz',
 'Genome31.fq.gz',
 'Genome41.fq.gz',
 'Genome51.fq.gz']

In [93]:
reads2

['Genome12.fq.gz',
 'Genome22.fq.gz',
 'Genome32.fq.gz',
 'Genome42.fq.gz',
 'Genome52.fq.gz']